In [111]:
import pandas as pd
from util.general import load_csv_pandas

In [112]:
# metropolitan = ['서울서부', '대전', '부산', '대구', '광주', '인천', '울산', '제주']

In [113]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = load_csv_pandas('원천조사가격정보_201601.csv')

# 소비자 가격만 남기고 다 없애기.
df = df[(df.조사구분명 == '소비자가격')].drop('조사구분명', axis=1)
df = df[['조사일자', '표준품목명', '조사가격품목명', '표준품종명', '조사가격품종명', '조사지역명', '표준시장명']]

In [114]:
tmp_df = df.groupby(["조사지역명", "표준품목명", "조사가격품목명", "조사가격품종명", "표준품종명"]).count()
print(tmp_df)

                                           조사일자  표준시장명
조사지역명 표준품목명    조사가격품목명 조사가격품종명   표준품종명                
강원    감        단감      일반        부유          14     14
      감귤       감귤      일반        조생귤         20     20
      감자       감자      수미        수미          20     20
      건가공(해조)류 건미역     일반        자연미역        20     20
      고구마      고구마(밤)  고구마(일반)   밤고구마        20     20
      고등어류     고등어     염장        망치고등어       18     18
      고추       붉은고추    일반        홍고추         20     20
               풋고추     꽈리고추      꽈리고추        20     20
                       일반고추      청초          20     20
                       청양고추      청양          20     20
      굴류       굴       일반        굴(참굴)       20     20
      깻잎       깻잎      일반        깻잎(일반)      20     20
      당근       당근      무세척       흙당근         20     20
      대파       파       대파        대파(일반)      20     20
      돈육       돼지고기    삼겹살(국산냉장) 삼겹살         20     20
      딸기       딸기      일반        설향          20     20
      땅콩  

In [115]:
l_df = tmp_df.groupby(["조사지역명"]).count()
# s_df = tmp_df.groupby(["조사지역명"]).sum()
print(l_df)

       조사일자  표준시장명
조사지역명             
강원       54     54
강원춘천     64     64
경기       60     60
경기수원     74     74
경기의정부    66     66
경남창원     78     78
경북안동     61     61
경북포항     72     72
광주       77     77
대구       78     78
대전       81     81
부산       81     81
서울       74     74
서울서부     78     78
울산       64     64
인천       78     78
전남순천     66     66
전북전주     74     74
전주       60     60
제주       77     77
청주       57     57
충북청주     70     70


In [116]:
# 서울, 경기도, 강원도, 경상도, 전라도, 충청도, (제주도)